# Spectral Relaxation for K-means Clustering - Code

Josipa Radnić, Tea Maričić, Lara Milić, Eleonora Detić

#### 1. Matrična formulacija k sredina

Prvo ćemo presložiti matricu podataka(**A**) na način da združimo one elemente koji pripadaju istom klasteru i napravimo od njih blokove. Nakon toga, pomoću kreiranih blokova, računamo broj elementa klastera(**$s_i$**) i nove centroide (**$m_i$**). Potom kreiramo matricu X pomoću jediničnih vektora i $s_i$ na način koji smo opisali na predavanjima. Na kraju, preostaje ponovo inverzno djelovati istom permutacijom na matricu $X$ kako bismo dobili $\tilde{X}$ koja sadrži informacije o pripadnosti elementa početnoj particiji.

In [1]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

def matrix_formulation_Kmeans(A, D, k): #A is data matrix, D is list of correct clasters for columns of A, k is number of clasters
    
    new_A = np.empty([A.shape[0],0])
    s_i = np.empty(k, dtype=int) 
    m_i = np.empty([A.shape[0],k])
    permutation = np.empty(0,) 

    for i in range(k):
        A_i = A[:, np.where(D == i)[1]] 
        permutation = np.append(permutation, np.where(D == i)[1], axis=0) 
        new_A = np.append(new_A, A_i, axis=1)
        s_i[i] = np.where(D == i)[1].shape[0]
        e = np.ones(s_i[i])
        m_i[:,i] = (1/s_i[i]) * np.matmul(A_i, e) 
    
    X = np.zeros([A.shape[1],k])

    for i in range(k):
        if( i == 0):
            X[i:s_i[i], 0] = 1/s_i[1]
        else:
            X[ s_i[0:i].sum() : s_i[0:i].sum() + s_i[i], i] = 1/s_i[i-1]

    X_tilde = np.zeros([A.shape[1],k])

    for i in range(X.shape[0]):
        index = list(permutation).index(i)  
        X_tilde[int(permutation[i]), :] = X[i, :] 

    start_partition = np.empty(D.shape[1], dtype=int)

    for i in range(X.shape[0]):
        start_partition[i] = int(np.where(X_tilde[i, :] != 0)[0])
    
    start_partition = np.reshape(start_partition, (D.shape[1],1))
    accuracy = accuracy_score(start_partition, np.transpose(D))
    return accuracy

#### 2.1 K-means algoritam 1

U ovom poglavlju navodimo učinkovitu verziju k-means algoritma za klasteriranje koristeći metode linearne algebre.

In [7]:
def K_means_1(A_train, A_test, D_train, D_test, k): 
    #A_train and A_test are data matrix, D_train and D_test are arrays, k is number of clasters
    
    base_for_k_elements = dict()
    
    for i in range (k):
        ii = [j for j, x in enumerate(D_train) if x == i] 
        B = A_train[:,ii]
        U, S, V = np.linalg.svd(B)
        base_for_k_elements[i] = U[:, 0:k]
    
    solution = np.zeros(len(D_test))
    
    for i in range (len(D_test)):
        min_distance = np.inf 
        claster = 0
        for j in range (0,k):
            distance = np.linalg.norm(A_test[:,i] - np.dot(np.dot(base_for_k_elements[j], base_for_k_elements[j].T),A_test[:,i]))
            if (distance < min_distance):
                min_distance = distance
                claster = j 
        solution[i] = claster
    
    return accuracy_score(solution, D_test)

#### 2.2 K-means algoritam 2

U ovom poglavlju navodimo jednostavnu, standardnu verziju k-means algoritma koja klasterira podatke.

In [9]:
def K_means_2(A, starting_partition, k, epsilon): 
    #A is data frame, one data is one present as one column
    #starting_partition is starting partition, usually random
    #k is number of clasters
    #epsilon is small number, stop criterion
    

    A.columns = starting_partition
    partition = starting_partition

    number_of_iteration = 0
    Q_of_partition = np.inf

    while(Q_of_partition > epsilon):
    
        number_of_iteration += 1

        s_i = np.unique(partition, axis=0, return_counts=True)[1]
        a_i = A.groupby(level=0,axis=1).sum().add_suffix('. centroid')
        m_i = a_i / s_i 
    
        Q_of_partition_before = 0
        for i in range(A.shape[1]):
            Q_of_partition_before  += np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,A.columns[i]]))
        
        
        for i in range(A.shape[1]): 
            distance_final = np.inf
            for j in range(0, m_i.shape[1]):
                distance = np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,j]))
                if(distance < distance_final):
                    distance_final = distance
                    tmp = list(A.columns)
                    tmp[i] = j
                    A.columns = tmp 
                
        partition = A.columns.values
    
        Q_of_partition_after = 0
        for i in range(A.shape[1]):
            Q_of_partition_after  += np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,A.columns[i]]))
    
        Q_of_partition = Q_of_partition_before - Q_of_partition_after

    return number_of_iteration, m_i, A.columns


#### 3.1 Spektralna relaksacija

Koristeći spektralnu relaksaciju, cilj je smanjiti  dimenzije matrice A, ali svejedno očuvati sve potrebne informacije. Koristeći rezulate članka, vidimo da je dovoljno uzeti samo prvih k svojstvenih vektora koji odgovaraju k najvećim svojstvenim vrijednostima. Na taj način formiramo matricu X_k na čije retke primjenjujemo k-means. Svaki podatak više nije $m \times 1$, već $k \times 1$.

In [14]:
from scipy.linalg import eigh
import numpy as np

def spectral_relaxation(A):
    #A is data matrix

    eigenvalues, eigenvectors = eigh(np.transpose(A) @ A) 
    
    eigenvalues = eigenvalues[A.shape[1]-k:A.shape[1]][::-1] 
    X_k = np.flip(eigenvectors[:, (A.shape[1]-k):A.shape[1]] ,axis=1)
    
    return X_k

#### 3.2 QR faktorizacija

Daljni pokušaji poboljšanja k-means algoritma tiču se početne particije. Naime, voljeli bismo nekako dobiti najbolju moguću početnu particiju. Prije je bila ona bila random, sada bismo je voljeli nekako specificirati i reći da imamo najbolju moguću. Šta god "najbolje moguće" značilo.

In [16]:
def QR_for_starting_partition(A):
    #A is data matrix
    
    X_k = spectral_relaxation(A)

    Q, R, P = scipy.linalg.qr(np.transpose(X_k), pivoting = True)
    P_ = np.zeros([A.shape[1],A.shape[1]])
    for i in range(A.shape[1]):
        P_[i,P[i]] = 1

    R_11 = R[0:k, 0:k]
    R_12 = R[0:k, k:R.shape[1]]
    I = np.eye(k, dtype=int)
    R = np.matmul(np.linalg.inv(R_11), R_12)

    R_kapa = np.append(I,R,axis=1)  @  np.transpose(P_)
    R_kapa = np.absolute(R_kapa)
    
    return np.argmax(R_kapa, axis=0)

#### 4. 1 K-means koristeći spektralnu relaksaciju

In [25]:
def p_K_means(A,k,epsilon):
    
    X_k = spectral_relaxation(A)
    X_k = np.transpose(X_k)
    
    while True: 
        starting_partition = np.random.randint(0, k, A.shape[1])
        s_i = np.unique(starting_partition, axis=0, return_counts=True)[1]
        if len(s_i) == k:
            break
    
    return K_means_2(pd.DataFrame(X_k), starting_partition, k, epsilon)
    

#### 4.2 K-means koristeći QR

In [26]:
def p_QR(A,k,epsilon):
    
    X_k = spectral_relaxation(A)
    X_k = np.transpose(X_k)
    
    starting_partition = QR_for_starting_partition(A)
    
    return K_means_2(pd.DataFrame(X_k), starting_partition, k, epsilon)

#### 4.3 K-means for predict

#### 5.1 Podaci - znamenke

In [2]:
import scipy.io

mat_1 = scipy.io.loadmat('azip.mat')
A_digits = mat_1['azip'] 

mat_2 = scipy.io.loadmat('dzip.mat')
D_digits = mat_2['dzip']

k_digits = 10 

#### 5.2 Podaci - pacijenti

#### 5.3 Podaci - odjevni predmeti

#### 5.4 Podaci - točke

#### 6. Usporedba rezultata, zaključci